<a href="https://colab.research.google.com/github/parthasarathydNU/gen-ai-coursework/blob/main/advanced-llms/direct-preference-optimization/DPODataSetGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# List files to identify what needs to be deleted
!ls -lh

# Remove specific files
!rm my_large_file.zip

# Clear pip cache
!rm -rf ~/.cache/pip

# Clear temporary files
!rm -rf /tmp/*

# Find and remove large files
!du -h /content | sort -hr | head -n 20
!rm /content/path/to/largefile

# Unmount Google Drive if mounted
from google.colab import drive
drive.flush_and_unmount()

# Clear datasets cache
import shutil
datasets_cache_dir = "/root/.cache/huggingface/datasets"
shutil.rmtree(datasets_cache_dir, ignore_errors=True)

# Clear transformers cache
transformers_cache_dir = "/root/.cache/huggingface/transformers"
shutil.rmtree(transformers_cache_dir, ignore_errors=True)




total 4.0K
drwxr-xr-x 1 root root 4.0K Aug  1 13:24 sample_data
rm: cannot remove 'my_large_file.zip': No such file or directory
55M	/content/sample_data
55M	/content
128K	/content/.config
84K	/content/.config/logs
80K	/content/.config/logs/2024.08.01
8.0K	/content/.config/configurations
rm: cannot remove '/content/path/to/largefile': No such file or directory
Drive not mounted, so nothing to flush and unmount.


In [ ]:
# Restart the runtime
from google.colab import runtime
runtime.unassign()

In [1]:
import gc

In [2]:
gc.collect()

23

In [3]:
!pip install git+https://github.com/huggingface/transformers -q peft  accelerate bitsandbytes safetensors sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 20.7 MB/s eta 0:00:00


In [4]:
!git config --global credential.helper store

# Install the huggingface_hub library
!pip install huggingface_hub -q

In [7]:
# Import the notebook_login method
from huggingface_hub import notebook_login

# Log in interactively
notebook_login()

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = 'mistralai/Mistral-7B-Instruct-v0.2'

def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )

    return model

def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer


model = load_quantized_model(model_name)

tokenizer = initialize_tokenizer(model_name)

# Define stop token ids
stop_token_ids = [0]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
def generate_response(prompt):
  text = f"[INST] {prompt} [/INST]"
  encoded = tokenizer(text, return_tensors="pt", add_special_tokens=False)
  model_input = encoded.to(model.device)
  generated_ids = model.generate(**model_input, max_new_tokens=200, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded[0].replace(text, '').strip()

In [7]:
prompt = "How AI will replace Engineers"
response = generate_response(prompt)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


It's important to QUALIFY the statement before providing an answer. AI and automation will not completely replace engineers in the same way that cars didn't replace horseback travel, but instead will complement and transform their roles. Engineers will continue to be needed for designing, building, maintaining, and improving AI systems. While AI can perform certain repetitive or standard tasks, it lacks the ability to invent new technologies, make ethical decisions, and consider the complex, real-world implications of engineering projects. Engineers will continue to be essential in applying the power of AI to create new solutions and drive technological innovations.</s>


In [12]:
!pip install datasets -q --no-cache

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 185.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 202.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 213.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 167.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 202.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 208.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2024.5.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have

In [16]:
!pip uninstall pyarrow -y

Found existing installation: pyarrow 17.0.0
Uninstalling pyarrow-17.0.0:
  Successfully uninstalled pyarrow-17.0.0


In [14]:
!pip install --upgrade pyarrow

In [19]:
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [1]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("GAIR/lima")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

In [2]:
import random

# Extract conversations from the train subset
conversations = ds['train']['conversations']

# Sample 50 random instructions
sampled_conversations = random.sample(conversations, 50)

# Display the sampled conversations
for i, conversation in enumerate(sampled_conversations, 1):
    print(f"{i}. {conversation[0]}")

1. When is the best time to rob a convenience store
2. Here is a newsflash I just got:
> Spontaneous riots at night in Tel Aviv following the firing of Defense Minister Yoav Gallant.
What questions should I be asking to better understand the situation?
3. Am I the asshole for not telling my girlfriend that my parents are gay?
4. Suppose I have the geographic coordinates of &quot;Saratoga, California, USA&quot; as
```Latitude:   37°15.8298′ N
Longitude: 122° 1.3806′ W
```
I know from here that in the case of latitude ```1° ≈ 69 miles``` and that longitude varies:
```1° longitude = cosine (latitude) * length of degree (miles) at Equator.
```
How many miles is 1° longitude at ```longitude: 122°1.3806′ W```?
5. I feel chest pain, what should I do?
6. O que é Reflection. Por que é útil?
* É recomendável usar em projetos?
* Como usar?
* Em quais situações Reflection pode ser usado?
7. How to make a tube top?
8. Since I created my repository it appears that the tags I have been
creating are n

In [8]:
# prompt: # For each of the above conversation, run the generate_response function 5 times
# # store this as a json of conversation number and array of generated responses

import json

results = {}
for i, conversation in enumerate(sampled_conversations, 1):
    responses = []
    for _ in range(5):
        response = generate_response(conversation[0])
        responses.append(response)
    results[i] = responses

# Save results as JSON
with open('generated_responses.json', 'w') as f:
    json.dump(results, f, indent=4)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [9]:
from google.colab import files

# Download the zipped file
files.download('generated_responses.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>